<a href="https://colab.research.google.com/github/xiaofanpaiooo-code/whisper-lora/blob/xiaofanpaiooo-code-patch-1/whisper-2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# 部署说明：
# 1. 复制此代码到 Colab。
# 2. 运行。
# 3. 这次使用了 Popen 管道直连，必须能看到输出！
# ==========================================

import os
import sys
import subprocess
import time

# 1. 环境安装
def install_dependencies():
    packages = ["faster-whisper", "fastapi", "uvicorn", "python-multipart", "pyngrok"]
    # -q 表示静默安装
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)

try:
    import faster_whisper
    import fastapi
    from pyngrok import ngrok
    import uvicorn
except ImportError:
    print("正在安装依赖环境，请稍候...", flush=True)
    install_dependencies()
    print("环境安装完成！", flush=True)

# ================= 配置区域 =================
NGROK_AUTH_TOKEN = "37hXaj4l3VhFmrso4FTbEWYE3Li_7Z3vCLhdMAPb2BpiVufAw"
# ===========================================

# 2. 生成独立服务脚本 (包含端口清理功能)
server_code = f"""
import os
import sys
import shutil
import time
import subprocess
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from faster_whisper import WhisperModel
from pyngrok import ngrok
import uvicorn
import torch

# 强制刷新打印函数
def log(msg):
    print(msg, flush=True)

# 杀掉占用 8000 端口的旧进程
def kill_port(port):
    try:
        command = f"fuser -k {{port}}/tcp"
        subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        time.sleep(1)
    except:
        pass

NGROK_AUTH_TOKEN = "{NGROK_AUTH_TOKEN}"

kill_port(8000)
app = FastAPI()

# 检查 GPU
if not torch.cuda.is_available():
    log("\\n❌ 严重错误：未检测到 GPU！")
    log("请在顶部菜单点击：[代码执行程序] -> [更改运行时类型] -> 选择 T4 GPU\\n")
    sys.exit(1)

log("正在加载 Whisper 模型 (Tiny + float16)...")
try:
    model = WhisperModel("tiny", device="cuda", compute_type="float16")
    log("✅ 模型加载完毕！")
except Exception as e:
    log(f"❌ 模型加载失败: {{e}}")
    sys.exit(1)

@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    start_time = time.time()
    temp_filename = f"temp_{{file.filename}}"
    try:
        with open(temp_filename, "wb") as buffer:
            shutil.copyfileobj(file.file, buffer)

        segments, info = model.transcribe(temp_filename, beam_size=1, language="zh")
        text_result = "".join([segment.text for segment in segments])

        log(f"耗时: {{time.time() - start_time:.2f}}s | 结果: {{text_result}}")
        return JSONResponse(content={{"text": text_result.strip()}})
    except Exception as e:
        log(f"❌ 推理出错: {{e}}")
        return JSONResponse(content={{"error": str(e)}}, status_code=500)
    finally:
        if os.path.exists(temp_filename):
            os.remove(temp_filename)

if __name__ == "__main__":
    if not NGROK_AUTH_TOKEN:
        log("❌ 错误：Token 未设置")
    else:
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)
        try:
            public_url = ngrok.connect(8000).public_url
            log(f"\\n======== 服务已启动 ========")
            log(f"API 接口地址 (请复制到客户端): {{public_url}}/transcribe")
            log(f"===========================\\n")
            uvicorn.run(app, host="0.0.0.0", port=8000)
        except Exception as e:
            log(f"❌ 启动失败: {{e}}")
"""

with open("server_main.py", "w", encoding="utf-8") as f:
    f.write(server_code)

print("已生成服务脚本，正在启动...", flush=True)

# 3. 使用 Popen + 实时管道读取 (终极解决方案)
# 这能确保哪怕是一个字符的输出，也能立刻被抓取显示
env = os.environ.copy()
env["PYTHONUNBUFFERED"] = "1"

process = subprocess.Popen(
    [sys.executable, "server_main.py"],
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT, # 把错误信息也合并到输出流
    text=True,
    bufsize=1 # 行缓冲
)

# 实时读取输出并打印
try:
    for line in process.stdout:
        print(line, end="")
except KeyboardInterrupt:
    print("服务已停止")
    process.terminate()

已生成服务脚本，正在启动...
正在加载 Whisper 模型 (Tiny + float16)...
✅ 模型加载完毕！

======== 服务已启动 ========
API 接口地址 (请复制到客户端): https://unavoidable-amber-plinthlike.ngrok-free.dev/transcribe

INFO:     Started server process [12274]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
耗时: 2.92s | 结果: 這才就有個影片已有機會做到以後我才跟過了但從 2009年大陸的年輕人會打到此時的大陸獵人一般的一般一般的一般一般的一般一般的一般一般的一般
INFO:     64.181.228.168:0 - "POST /transcribe HTTP/1.1" 200 OK
耗时: 3.00s | 结果: 在這裏的風 跟著其他人的大人很適合在這裏的小子和小子的小子的小子和小子的小子的小子的小子和小子的小子的小子的小子
INFO:     64.181.228.168:0 - "POST /transcribe HTTP/1.1" 200 OK
耗时: 0.72s | 结果: 本身是Movie Bronson 2009所以其實其實其實是Cling Charles Bronson
INFO:     64.181.228.168:0 - "POST /transcribe HTTP/1.1" 200 OK
耗时: 2.10s | 结果: 突然是能發現他喜歡蜜蜜就是一位康廷的朋友和光珍紋的時間就算好好玩
INFO:     64.181.228.168:0 - "POST /transcribe HTTP/1.1" 200 OK
耗时: 1.49s | 结果: 他有一個人的心情他在一邊都在這兒他在這兒都在這兒他在這兒都在這兒然後他在這兒他在這兒都在這兒他在這兒都在這兒
INFO:     64.181.228.168:0 - "